# 🧠 MedFlow AI — A multi-agent clinical intelligence system

MedFlow AI is a human-in-the-loop, agent-orchestrated clinical assistant built on Google MedGemma.
The system separates documentation, clinical reasoning, and decision-making to ensure safety, transparency, and medical compliance.

🧩 System Overview

MedFlow AI consists of two MedGemma-powered agents coordinated by an orchestrator and supervised by a licensed doctor.

* Agent 1: Clinical Documentation (SOAP: S, O, A)

* Agent 2: Plan Analysis, Labs & Lifestyle Guidance

* Doctor: Final authority and decision-maker

## Model Used

google/medgemma-1.5-4b-it

Chosen for:

* Strong medical language understanding

* Safe clinical summarization

* Structured output generation

In [1]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from transformers import pipeline
import torch
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)


MODEL_ID = "google/medgemma-1.5-4b-it"

pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda",
)

2026-02-09 05:31:44.495064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770615104.684002      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770615104.739230      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770615105.202672      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770615105.202709      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770615105.202712      55 computation_placer.cc:177] computation placer alr

config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda


## 🤖 Agent 1 — SOAP Note Generator
📌 Purpose

Agent 1 converts raw patient input into a structured clinical SOAP note, focusing strictly on documentation — not decision-making.

This agent does NOT:

* Diagnose diseases

* Prescribe medications

* Suggest treatment plans

In [2]:
from transformers import pipeline
from PIL import Image
import requests
import json
import torch
import re


# ===== Agent 1: generate S/O/A =====
def run_agent_1(patient_info: dict, images: list = None):
    """
    Generates Subjective, Objective, Assessment only from patient info and optional images.
    Returns a dict with keys: subjective, objective, assessment, missing_information, safety_notice
    """
    
    # Build MedGemma chat messages
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": 
                         """You are Agent 1 in the MedFlow AI system.

Your role is to collect, validate, clean, and structure raw user input before it is passed to downstream agents.

Responsibilities:
- Accept raw input text or extracted content from speech or documents
- Detect missing, unclear, or conflicting information
- Normalize language, spelling, and formatting
- Extract key medical or domain-relevant entities
- Convert unstructured input into clean, structured JSON
- Add flags for ambiguity or low-confidence data
- Do NOT make diagnoses or recommendations

Rules:
- Stay strictly within data processing and structuring
- Do not infer beyond the provided input
- If information is missing, explicitly flag it
- Preserve the original meaning of the input

Output:
- Return only valid JSON
- Include structured data, flags, and confidence score
"""
                        }]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": (
                "Generate a structured SOAP note in JSON format containing ONLY "
                "Subjective (S), Objective (O), and Assessment (A). "
                "Do NOT include Plan, Diagnosis, or Medications. "
                "Use neutral clinical language. "
                "If any information is missing, list it under 'missing_information'. "
                "Add a 'safety_notice' field with precautions.\n\n"
                f"Patient data:\n{json.dumps(patient_info, indent=2)}"
            )}]
        }
    ]
    
    # Attach images if provided
    if images:
        for img in images:
            messages[1]["content"].append({"type": "image", "image": img})
    
    # Generate output
    output = pipe(text=messages, max_new_tokens=800)
    assistant_text = output[-1]["generated_text"]

    # Remove markdown ```json if present
    json_text = re.sub(r"```json|```", "", assistant_text[-1]["content"]).strip()
    
    # Parse JSON safely
    try:
        data = json.loads(json_text)
    except json.JSONDecodeError:
        data = {
            "subjective": "",
            "objective": "",
            "assessment": "",
            "missing_information": ["Patient vitals or history may be incomplete."],
            "safety_notice": "Unable to generate full SOAP note. Please verify patient data."
        }
    
    # Standardize keys
    return {
        "subjective": data.get("S", data.get("subjective", "")),
        "objective": data.get("O", data.get("objective", "")),
        "assessment": data.get("A", data.get("assessment", "")),
        "missing_information": data.get("missing_information", []),
        "safety_notice": data.get("safety_notice", "")
    }



## Example Usage

* Patient age & gender

* Symptoms

* Duration & severity

* Medical history

* Vitals (if available)

In [3]:
# Optional: add image
image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

patient_input = {
    "age": 45,
    "gender": "Male",
    "symptoms": [
        "Chest discomfort",
        "Shortness of breath during exertion",
        "Fatigue"
    ],
    "duration": "2 weeks",
    "severity": "Moderate",
    "medical_history": ["Hypertension"],
    "medications": [],
    "vitals": {
        "blood_pressure": "145/90",
        "heart_rate": "92 bpm"
    }
}

### Example Usage

In [4]:
soap_note = run_agent_1(patient_input, images=[image])
print("=== SOAP S/O/A ===")
print(json.dumps(soap_note, indent=2))

=== SOAP S/O/A ===
{
  "subjective": {
    "chief_complaint": "Chest discomfort, shortness of breath during exertion, and fatigue.",
    "history_of_present_illness": "Patient reports experiencing chest discomfort, shortness of breath during exertion, and fatigue for the past 2 weeks. The symptoms are described as moderate in severity.",
    "past_medical_history": "Patient has a history of hypertension.",
    "medications": [],
    "allergies": [],
    "social_history": "Missing",
    "family_history": "Missing",
    "review_of_systems": "Missing",
    "missing_information": [
      "Social history",
      "Family history",
      "Review of systems",
      "Detailed description of chest discomfort (location, character, radiation)",
      "Details about shortness of breath (onset, triggers, relieving factors)"
    ]
  },
  "objective": {
    "vital_signs": {
      "blood_pressure": "145/90 mmHg",
      "heart_rate": "92 bpm",
      "respiratory_rate": "Missing",
      "temperature": "M

## 🤖 Agent 2 — Plan, Labs & Lifestyle Analyzer
📌 Purpose

Agent 2 evaluates the doctor-provided plan in the context of:

* Symptoms

* Assessment


Use MedGemma to provide supportive, explainable insights, not decisions.

In [5]:
def run_agent_2(soap_note: dict, doctor_plan: dict, ethnicity: str = "Not provided"):
    messages = [
        {
            "role": "system",
            "content": [{
                "type": "text",
                "text": (
                    """You are Agent 2 in the MedFlow AI system.

Your role:
- Receive the SOAP note (S, O, A) generated by Agent 1.
- Receive the Plan provided by the doctor (medications, tests, follow-up).
- Produce a complete SOAP note including Subjective, Objective, Assessment, and the Plan.
- Analyze the plan for alignment with symptoms and assessment.
- Provide recommendations for lifestyle, food, exercise, clothing, music, and fragrances based on patient context and prescription.
- Highlight missing information or caution if relevant.
- Maintain patient safety and clinical neutrality.
- Do NOT diagnose or prescribe new medications or new tests.

Output format:
- Return valid JSON ONLY.
- Include the following keys:
  1. soap_note: a full SOAP note including Plan
  2. medication_review: alignment score (%) and rationale
  3. test_validation: relevance score (%) and rationale for each test
  4. lifestyle_recommendations: food, exercise, clothing, music, fragrance
  5. additional_notes: optional notes on patient context or missing info
- Percentages should be numbers (0-100)
- Explain reasoning in rationale fields
- Clearly indicate missing or uncertain information

"""
                )
            }]
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": (
                    "Analyze the following clinical data.\n\n"
                    "Tasks:\n"
                    "1. Evaluate whether the prescribed medicines align with the symptoms and assessment.\n"
                    "2. Evaluate whether the prescribed lab tests are clinically relevant.\n"
                    "3. Provide confidence scores (0–100%) with brief rationales.\n"
                    "4. Suggest lifestyle, food, exercise, clothing, music, and fragrance recommendations.\n\n"
                    "Return ONLY a JSON object.\n\n"
                    f"SOAP Note:\n{json.dumps(soap_note, indent=2)}\n\n"
                    f"Doctor Plan:\n{json.dumps(doctor_plan, indent=2)}\n\n"
                    f"Patient Ethnicity: {ethnicity}"
                )
            }]
        }
    ]

    output = pipe(text=messages, max_new_tokens=2000)
    assistant_text = output[-1]["generated_text"]

    json_text = re.sub(r"```json|```", "", assistant_text[-1]["content"]).strip()

    try:
        return json.loads(json_text)
    except json.JSONDecodeError:
        return {
            "medicine_alignment": {"confidence_score": None, "rationale": "Parsing failed"},
            "lab_test_analysis": [],
            "lifestyle_recommendations": {},
            "missing_information": ["Model output could not be parsed"],
            "safety_notice": "Consult a healthcare professional."
        }

## 👨‍⚕️ Doctor-in-the-Loop (Critical Step)

Before any recommendations are made:

*  Doctor reviews SOAP (S/O/A)

*  Doctor adds Plan (P) manually

*  Doctor may prescribe:

* Medications

* Lab tests

* Follow-up instructions

👉 No AI output is patient-facing without doctor approval

## Example Usage

Doctor adds medications, lab tests and ethnicity for Soap note and AI suggestions.

In [6]:
# ===== Doctor adds the plan =====
patient_ethnicity = "South Asian"
doctor_plan = {
    "medications": ["Omeprazole 20mg once daily"],
    "lab_tests": ["H. pylori test", "CBC"],
    "follow_up": "2 weeks"
}


{
  "soap_note": {
    "subjective": {
      "chief_complaint": "Chest discomfort, shortness of breath during exertion, and fatigue.",
      "history_of_present_illness": "Patient reports experiencing chest discomfort, shortness of breath during exertion, and fatigue for the past 2 weeks. The symptoms are described as moderate in severity.",
      "past_medical_history": "Patient has a history of hypertension.",
      "medications": [],
      "allergies": [],
      "social_history": "Missing",
      "family_history": "Missing",
      "review_of_systems": "Missing",
      "missing_information": [
        "Social history",
        "Family history",
        "Review of systems",
        "Detailed description of chest discomfort (location, character, radiation)",
        "Details about shortness of breath (onset, triggers, relieving factors)"
      ]
    },
    "objective": {
      "vital_signs": {
        "blood_pressure": "145/90 mmHg",
        "heart_rate": "92 bpm",
        "respiratory

In [ ]:
# Agent 2 generates the soap note and other recommendations.
agent2_output = run_agent_2(soap_note, doctor_plan, patient_ethnicity)
print(json.dumps(agent2_output, indent=2))

## Visualize JSON and PDF Generation
Paste the agent2_output in JSONCRACK to visualize it.

In [14]:
!pip install reportlab -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import json

from IPython.display import IFrame, display
import urllib.parse

json_str = json.dumps(agent2_output["soap_note"], indent=2)
encoded_json = urllib.parse.quote(json_str)

url = f"https://jsoncrack.com/editor?json={encoded_json}"

display(IFrame(src=url, width=1200, height=700))

### Soap note pdf
SOAP note is generated. Can be modified according to the need.

In [18]:
#!/usr/bin/env python3
"""
Medical SOAP Note Generator
Creates a professional medical SOAP note in PDF format following standard medical documentation practices.
"""

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_JUSTIFY
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib import colors
from datetime import datetime
import json

# JSON data embedded directly


# Create PDF
pdf_filename = './medical_soap_note.pdf'
doc = SimpleDocTemplate(pdf_filename, pagesize=letter,
                        topMargin=0.5*inch, bottomMargin=0.5*inch,
                        leftMargin=0.75*inch, rightMargin=0.75*inch)

# Container for the 'Flowable' objects
elements = []

# Get standard styles
styles = getSampleStyleSheet()

# Define custom styles
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=16,
    textColor=colors.HexColor('#1a1a1a'),
    spaceAfter=6,
    alignment=TA_CENTER,
    fontName='Helvetica-Bold'
)

header_style = ParagraphStyle(
    'SectionHeader',
    parent=styles['Heading2'],
    fontSize=12,
    textColor=colors.HexColor('#2c3e50'),
    spaceAfter=8,
    spaceBefore=12,
    fontName='Helvetica-Bold',
    borderWidth=1,
    borderColor=colors.HexColor('#2c3e50'),
    borderPadding=4,
    backColor=colors.HexColor('#ecf0f1')
)

subheader_style = ParagraphStyle(
    'SubHeader',
    parent=styles['Heading3'],
    fontSize=10,
    textColor=colors.HexColor('#34495e'),
    spaceAfter=4,
    spaceBefore=6,
    fontName='Helvetica-Bold'
)

body_style = ParagraphStyle(
    'CustomBody',
    parent=styles['Normal'],
    fontSize=10,
    textColor=colors.HexColor('#2c3e50'),
    spaceAfter=6,
    alignment=TA_JUSTIFY,
    fontName='Helvetica'
)

alert_style = ParagraphStyle(
    'Alert',
    parent=styles['Normal'],
    fontSize=10,
    textColor=colors.HexColor('#c0392b'),
    spaceAfter=6,
    fontName='Helvetica-Bold',
    backColor=colors.HexColor('#fadbd8'),
    borderWidth=1,
    borderColor=colors.HexColor('#c0392b'),
    borderPadding=6
)

# Title
elements.append(Paragraph("MedFlow AI", title_style))
elements.append(Paragraph("SOAP NOTE", ParagraphStyle('Subtitle', parent=styles['Heading2'], 
                                                       fontSize=14, alignment=TA_CENTER, 
                                                       textColor=colors.HexColor('#1a1a1a'),
                                                       fontName='Helvetica-Bold', spaceAfter=6)))
elements.append(Spacer(1, 0.1*inch))

# Patient Information Header
patient_info_data = [
    ['Date:', datetime.now().strftime('%B %d, %Y')],
    ['Patient ID:', '[Patient ID]'],
    ['Provider:', '[Provider Name]']
]

patient_table = Table(patient_info_data, colWidths=[1.5*inch, 4*inch])
patient_table.setStyle(TableStyle([
    ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
    ('FONTSIZE', (0, 0), (-1, -1), 9),
    ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.HexColor('#2c3e50')),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('VALIGN', (0, 0), (-1, -1), 'TOP'),
]))
elements.append(patient_table)
elements.append(Spacer(1, 0.2*inch))

# ============= SUBJECTIVE =============
elements.append(Paragraph("S — SUBJECTIVE", header_style))
elements.append(Spacer(1, 0.1*inch))

# Chief Complaint
soap_data = agent2_output['soap_note']
elements.append(Paragraph("<b>Chief Complaint:</b>", subheader_style))
elements.append(Paragraph(soap_data['subjective']['chief_complaint'], body_style))

# History of Present Illness
elements.append(Paragraph("<b>History of Present Illness:</b>", subheader_style))
elements.append(Paragraph(soap_data['subjective']['history_of_present_illness'], body_style))

# Past Medical History
elements.append(Paragraph("<b>Past Medical History:</b>", subheader_style))
pmh_text = soap_data['subjective']['past_medical_history']
elements.append(Paragraph(pmh_text, body_style))

# Medications
elements.append(Paragraph("<b>Current Medications:</b>", subheader_style))
medications = soap_data['subjective']['medications']
if medications:
    med_text = ", ".join(medications)
    elements.append(Paragraph(med_text, body_style))
else:
    elements.append(Paragraph("None reported", body_style))

# Allergies
elements.append(Paragraph("<b>Allergies:</b>", subheader_style))
allergies = soap_data['subjective']['allergies']
if allergies:
    allergy_text = ", ".join(allergies)
    elements.append(Paragraph(allergy_text, body_style))
else:
    elements.append(Paragraph("No known drug allergies (NKDA)", body_style))

# Social History
social_history = soap_data['subjective'].get('social_history', '')
if social_history and social_history != "Missing":
    elements.append(Paragraph("<b>Social History:</b>", subheader_style))
    elements.append(Paragraph(social_history, body_style))

# Family History
family_history = soap_data['subjective'].get('family_history', '')
if family_history and family_history != "Missing":
    elements.append(Paragraph("<b>Family History:</b>", subheader_style))
    elements.append(Paragraph(family_history, body_style))

# Review of Systems
ros = soap_data['subjective'].get('review_of_systems', '')
if ros and ros != "Missing":
    elements.append(Paragraph("<b>Review of Systems:</b>", subheader_style))
    elements.append(Paragraph(ros, body_style))

elements.append(Spacer(1, 0.15*inch))

# ============= OBJECTIVE =============
elements.append(Paragraph("O — OBJECTIVE", header_style))
elements.append(Spacer(1, 0.1*inch))

# Vital Signs
elements.append(Paragraph("<b>Vital Signs:</b>", subheader_style))
vitals = soap_data['objective']['vital_signs']

vital_data = []
if vitals.get('blood_pressure') and vitals.get('blood_pressure') != "Missing":
    vital_data.append(['Blood Pressure:', vitals.get('blood_pressure')])
if vitals.get('heart_rate') and vitals.get('heart_rate') != "Missing":
    vital_data.append(['Heart Rate:', vitals.get('heart_rate')])
if vitals.get('respiratory_rate') and vitals.get('respiratory_rate') != "Missing":
    vital_data.append(['Respiratory Rate:', vitals.get('respiratory_rate')])
if vitals.get('temperature') and vitals.get('temperature') != "Missing":
    vital_data.append(['Temperature:', vitals.get('temperature')])
if vitals.get('oxygen_saturation') and vitals.get('oxygen_saturation') != "Missing":
    vital_data.append(['Oxygen Saturation:', vitals.get('oxygen_saturation')])

if vital_data:
    vital_table = Table(vital_data, colWidths=[2*inch, 3*inch])
    vital_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 9),
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('TEXTCOLOR', (0, 0), (-1, -1), colors.HexColor('#2c3e50')),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('ROWBACKGROUNDS', (0, 0), (-1, -1), [colors.white, colors.HexColor('#f8f9fa')])
    ]))
    elements.append(vital_table)
    elements.append(Spacer(1, 0.1*inch))

# Physical Examination
physical_exam = soap_data['objective'].get('physical_exam', '')
if physical_exam and physical_exam != "Missing":
    elements.append(Paragraph("<b>Physical Examination:</b>", subheader_style))
    elements.append(Paragraph(physical_exam, body_style))

# Imaging
imaging = soap_data['objective']['imaging']
has_imaging = False
imaging_content = []

if imaging.get('chest_xray') and imaging.get('chest_xray') != "Missing":
    has_imaging = True
    imaging_content.append(Paragraph(f"<b>Chest X-Ray:</b> {imaging['chest_xray']}", body_style))

other_imaging = imaging.get('other_imaging', '')
if other_imaging and other_imaging != 'Missing':
    has_imaging = True
    imaging_content.append(Paragraph(f"<b>Other Imaging:</b> {other_imaging}", body_style))

if has_imaging:
    elements.append(Paragraph("<b>Imaging Studies:</b>", subheader_style))
    for img in imaging_content:
        elements.append(img)

# Laboratory Results
lab_results = soap_data['objective'].get('laboratory_results', '')
if lab_results and lab_results != "Missing":
    elements.append(Paragraph("<b>Laboratory Results:</b>", subheader_style))
    elements.append(Paragraph(lab_results, body_style))

elements.append(Spacer(1, 0.15*inch))

# ============= ASSESSMENT =============
elements.append(Paragraph("A — ASSESSMENT", header_style))
elements.append(Spacer(1, 0.1*inch))
elements.append(Paragraph(soap_data['assessment'], body_style))

elements.append(Spacer(1, 0.15*inch))

# ============= PLAN =============
elements.append(Paragraph("P — PLAN", header_style))
elements.append(Spacer(1, 0.1*inch))
elements.append(Paragraph(soap_data['plan'], body_style))

# Lifestyle Recommendations
if 'lifestyle_recommendations' in agent2_output:
    elements.append(Spacer(1, 0.1*inch))
    elements.append(Paragraph("<b>Lifestyle Recommendations:</b>", subheader_style))
    lifestyle = agent2_output['lifestyle_recommendations']
    
    if lifestyle.get('food'):
        elements.append(Paragraph(f"<b>Dietary:</b> {lifestyle['food']}", body_style))
    if lifestyle.get('exercise'):
        elements.append(Paragraph(f"<b>Exercise:</b> {lifestyle['exercise']}", body_style))
    if lifestyle.get('clothing'):
        elements.append(Paragraph(f"<b>Clothing:</b> {lifestyle['clothing']}", body_style))
    if lifestyle.get('music'):
        elements.append(Paragraph(f"<b>Stress Management:</b> {lifestyle['music']}", body_style))
    if lifestyle.get('fragrance'):
        elements.append(Paragraph(f"<b>Environmental:</b> {lifestyle['fragrance']}", body_style))

# Additional Notes
if agent2_output.get('additional_notes'):
    elements.append(Spacer(1, 0.1*inch))
    elements.append(Paragraph("<b>Additional Notes:</b>", subheader_style))
    elements.append(Paragraph(agent2_output['additional_notes'], body_style))

# Safety Notice
elements.append(Spacer(1, 0.15*inch))
if agent2_output.get('safety_notice'):
    elements.append(Paragraph("<b>SAFETY ALERT</b>", alert_style))
    elements.append(Paragraph(agent2_output['safety_notice'], alert_style))

# Footer
elements.append(Spacer(1, 0.2*inch))
footer_text = """
<para alignment="center" fontSize="8" textColor="#7f8c8d">
<i>This SOAP note is for medical documentation purposes. All information should be verified and supplemented with complete clinical assessment.</i><br/>
<i>Document generated on: {}</i>
</para>
""".format(datetime.now().strftime('%B %d, %Y at %H:%M'))
elements.append(Paragraph(footer_text, body_style))

# Build PDF
doc.build(elements)

print(f"SOAP note created successfully: {pdf_filename}")
print("\nSummary:")
print(f"- Chief Complaint: {soap_data['subjective']['chief_complaint']}")
print(f"- Blood Pressure: {vitals.get('blood_pressure', 'Not recorded')}")
print(f"- Assessment: Patient presents with symptoms suggestive of possible cardiac or pulmonary etiology")
print(f"- Plan: Includes omeprazole 20mg daily, H. pylori test, CBC, and 2-week follow-up")

SOAP note created successfully: ./medical_soap_note.pdf

Summary:
- Chief Complaint: Chest discomfort, shortness of breath during exertion, and fatigue.
- Blood Pressure: 145/90 mmHg
- Assessment: Patient presents with symptoms suggestive of possible cardiac or pulmonary etiology
- Plan: Includes omeprazole 20mg daily, H. pylori test, CBC, and 2-week follow-up


In [19]:

from IPython.display import IFrame
IFrame(src=pdf_filename, width=800, height=600)
